In [27]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from PIL import Image, ImageOps
from skimage import feature

from collections import OrderedDict
import statistics

In [2]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
im = Image.open("./HiPass_sobel_Threshold.tif")
im_gray = ImageOps.grayscale(im)
imarray = np.array(im_gray)

In [4]:
im_gray.show()

In [5]:
X, Y = imarray.shape

In [6]:
X * Y

1228800

In [7]:
im.show()

In [8]:
0 in imarray[:2]

False

In [9]:
unvisited = set([(x, y) for x in range(2, X) for y in range(3, Y) if imarray[x, y] == 255])

def get_cardinal_nodes(node, X, Y):
    x = node[0]
    y = node[1]
    
    result = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
    
    return [(i, j) for (i, j) in result if (i >= 0 and i < X and j >= 0 and j < Y)]


components = {}
count = 0

while unvisited:
    current_node = unvisited.pop()
    
    queue = [current_node]
    
    min_x = current_node[0]
    max_x = current_node[0]
    min_y = current_node[1]
    max_y = current_node[1]
    
    while queue:
        next_node = queue.pop()
        
        potential_neighbor_nodes = get_cardinal_nodes(next_node, X, Y)
        
        for node in potential_neighbor_nodes:
            i, j = node
            
            if node in unvisited:
                unvisited.remove(node)
                
                min_x = min(i, min_x)
                max_x = max(i, max_x)
                min_y = min(j, min_y)
                max_y = max(j, max_y)
                
                queue.append(node)
                
    components[count] = {"min_x": min_x, "max_x": max_x, "min_y": min_y, "max_y": max_y}
    count += 1

In [10]:
components[0]

{'min_x': 436, 'max_x': 437, 'min_y': 677, 'max_y': 680}

In [58]:
components

{0: {'min_x': 99, 'max_x': 107, 'min_y': 582, 'max_y': 597},
 1: {'min_x': 402, 'max_x': 442, 'min_y': 926, 'max_y': 955},
 2: {'min_x': 127, 'max_x': 134, 'min_y': 1247, 'max_y': 1252},
 3: {'min_x': 312, 'max_x': 315, 'min_y': 240, 'max_y': 241},
 4: {'min_x': 697, 'max_x': 725, 'min_y': 448, 'max_y': 459},
 5: {'min_x': 236, 'max_x': 382, 'min_y': 188, 'max_y': 319},
 6: {'min_x': 576, 'max_x': 605, 'min_y': 276, 'max_y': 311},
 7: {'min_x': 622, 'max_x': 660, 'min_y': 302, 'max_y': 329},
 8: {'min_x': 409, 'max_x': 426, 'min_y': 1187, 'max_y': 1226},
 9: {'min_x': 374, 'max_x': 385, 'min_y': 1164, 'max_y': 1181},
 10: {'min_x': 190, 'max_x': 244, 'min_y': 393, 'max_y': 436},
 11: {'min_x': 116, 'max_x': 133, 'min_y': 624, 'max_y': 642},
 12: {'min_x': 526, 'max_x': 545, 'min_y': 637, 'max_y': 676},
 13: {'min_x': 46, 'max_x': 81, 'min_y': 890, 'max_y': 918},
 14: {'min_x': 529, 'max_x': 533, 'min_y': 412, 'max_y': 426},
 15: {'min_x': 91, 'max_x': 133, 'min_y': 693, 'max_y': 728},


In [35]:
# Create a dictionary with Bounding Box Areas as keys and counts of components with said area as values
area_filter = OrderedDict()

for dim in components.values():
    width, height = dim['max_x'] - dim['min_x'] + 1, dim['max_y'] - dim['min_y'] + 1
    area = width * height

    area_filter[area] = area_filter.get(area, 0) + 1

area_filter = OrderedDict(sorted(area_filter.items()))

area_filter


OrderedDict([(1, 2568),
             (2, 1141),
             (3, 384),
             (4, 474),
             (5, 60),
             (6, 384),
             (7, 11),
             (8, 212),
             (9, 110),
             (10, 120),
             (11, 3),
             (12, 227),
             (14, 39),
             (15, 113),
             (16, 76),
             (18, 99),
             (20, 93),
             (21, 46),
             (22, 6),
             (24, 104),
             (25, 37),
             (27, 14),
             (28, 61),
             (30, 73),
             (32, 40),
             (33, 17),
             (35, 33),
             (36, 64),
             (38, 1),
             (39, 6),
             (40, 44),
             (42, 36),
             (44, 25),
             (45, 22),
             (48, 47),
             (49, 16),
             (50, 15),
             (52, 4),
             (54, 23),
             (55, 20),
             (56, 33),
             (57, 1),
             (60, 28),
             

In [52]:
areas, counts = np.array(list(area_filter.keys())), np.array(list(area_filter.values()))

weighted_average = np.inner(areas, counts) / sum(counts)

weighted_average

86.98991371977154